**Exercise 1**
<br/>
Implement the final version of the authenticated DH protocol in Slide 16. (You can
use external libraries.)


In [ ]:
!pip install rsa

In [ ]:
from sympy import randprime, primitive_root, factorint, isprime
import rsa
import random


# define participant with  authenticated DH protocol
class DHParticipant:

    __name: str
    __private_key: int
    __public_key: int
    __shared_private_key: int
    __p: int

    # constructor of participant class
    def __init__(self, name: str):
        self.__name = name

    # return q as a prime which the biggest prime factor of p - 1
    def __generate_q(self, p: int):
        # find list factors of p - 1
        factors = factorint(p - 1)
        # filter prime number in factor list
        prime_factors = [factor for factor in factors if isprime(factor)]
        # return the biggest factor in prime factor list
        return max(prime_factors) if prime_factors else None

    # return r as a random number as:
    #  (r**2) mod p != 1
    #  (g**r) mod p != 1
    #  (g**r) mod p != p - 1
    def __generate_r(self, p: int, g: int):
        while True:
            r = random.randint(2, p - 2)
            if pow(r, 2, p) != 1 and pow(g, r, p) != 1 and pow(g, r, p) != p - 1:
                return r

    # sign a message with RSA algorithm for authentication purpose
    # return (rsa public key, signature)
    def __generate_authentication(self, msg: str):
        rsa_public_key, rsa_private_key = rsa.newkeys(2048)
        signature = rsa.sign(msg.encode("utf-8"), rsa_private_key, "SHA-256")

        return (rsa_public_key, signature)

    # verify a message with public key and signature using RSA algorithm
    # return an error if authentication failed
    def __verify_authentication(self, msg: str, authentications):
        (rsa_public_key, signature) = authentications

        return rsa.verify(msg.encode("utf-8"), signature, rsa_public_key)

    # generate s as the minimum size of p
    # means that if s= 4, p need at least 4 bits to present
    def generate_s(self):
        # limit min between 2**8 to 2**16
        s = random.randint(8, 16)

        print(self.__name, f"chose s as {s}")

        return s

    # generate parameters based on s
    # return necessary params, public key and authentication info using RSA algorithm
    def select_parameters(self, s: int):
        # select p as a prime with s bits
        p = randprime(2 ** (s - 1), 2**s)
        self.__p = p
        # calculate g as primitive root of p
        g = primitive_root(p)
        # generate q, r based on p, g
        q = self.__generate_q(p)
        r = self.__generate_r(p, g)

        print(f"{self.__name} chose p as {p}, q as {q}, r as {r} with generator as {g}")

        # select private key randomly from 1 to (q - 1)
        self_private_key = random.randint(1, q - 1)
        self.__private_key = self_private_key
        # calculate public of its own as (g**(private key)) mod p
        self_public_key = pow(g, self_private_key, p)
        self.__public_key = self_public_key

        print(f"{self.__name}'s private key is {self.__private_key}")
        print(f"{self.__name}'s public key is {self.__public_key}")

        # generate authentication message based on parameters generated
        message_auth = f"{g},{p},{q},{r},{self_public_key}"
        auth = self.__generate_authentication(message_auth)

        return (g, p, q, r), self_public_key, auth

    # receive params from partner, calculate public, private and shared secret key
    # return public key, authentication info using RSA algorithm
    def receive_initial_parameters(self, params):
        (g, p, q, r), public_key, auth = params
        self.__p = p
        # create authentication message to validate received params
        message_auth = f"{g},{p},{q},{r},{public_key}"
        # verify message, throw error if not valid
        self.__verify_authentication(message_auth, auth)
        print(f"{self.__name} received successfully")
        # generate private key based on received q
        # private key is randomly from 1 to (q - 1)
        self_private_key = random.randint(1, q - 1)
        self.__private_key = self_private_key
        # calculate public of its own as (g**(private key)) mod p
        self_public_key = pow(g, self_private_key, p)
        self.__public_key = self_public_key

        print(f"{self.__name}'s private key is {self.__private_key}")
        print(f"{self.__name}'s public key is {self.__public_key}")

        # shared secret between 2 participants calculated as ((g **(private key of p1) **(private key of p2))) mod p
        # or as (public key of partner **(secret key of its own)) mod p
        shared_secret = pow(public_key, self_private_key, p)
        self.__shared_private_key = shared_secret

        print(f"{self.__name}'s shared secret key is {self.__shared_private_key}")

        # generate authentication params for partner
        auth = self.__generate_authentication(f"{self_public_key}")

        return self_public_key, auth

    # receive authentication params from partner, calculate shared secret key
    def receive_partner_public_key(self, partner_public_key, auth_params):
        # verify authentication params from partner, then verify it
        # throw error if it's invalid
        self.__verify_authentication(f"{partner_public_key}", auth_params)

        print(f"{self.__name} received successfully")

        # shared secret between 2 participants calculated as ((g **(private key of p1) **(private key of p2))) mod p
        # or as (public key of partner **(secret key of its own)) mod p
        shared_secret = pow(partner_public_key, self.__private_key, self.__p)
        self.__shared_private_key = shared_secret

        print(f"{self.__name}'s shared secret key is {self.__shared_private_key}")


# initial 2 object Alice and Bob
alice = DHParticipant("Alice")
bob = DHParticipant("Bob")
# step 1: Alice select s as minimum size of the prime
s = alice.generate_s()
# step 2: Bob generate params based on s
params = bob.select_parameters(s)

try:
    # step 3: Alice receive the params from Bob, verify then calculate the shared secret for itself
    alice_public_key, auth_params = alice.receive_initial_parameters(params)
    # step 4: Bob receive the authentication params from Alice, verify then calculate the shared secret itself
    bob.receive_partner_public_key(alice_public_key, auth_params)
except:
    print("invalid signature!")

Alice chose s as 11
Bob chose p as 1231, q as 41, r as 1053 with generator as 3
Bob's private key is 5
Bob's public key is 243
Alice received successfully
Alice's private key is 9
Alice's public key is 1218
Alice's shared secret key is 469
Bob received successfully
Bob's shared secret key is 469


**Exercise 2**

Design and implement a simple key management protocol. The protocol should be
based on a KDC that shares keys with Alice and Bob. Alice, initiating communi-
cation, should establish (through the KDC) a shared key with Bob. Introduce the
following three classes: **KDC, Alice, Bob**, and present the protocol as an interac-
tion between three objects of these classes. Refer to the Kerberos protocol in Slide
20 as an example.

a) Are you aware of any limitations or security problems of your solution (consider replay attacks, confidentiality and authentication, overheads, etc.)?

b) Do you see any ways of improving them?

In [3]:
!pip install pycryptodome

In [16]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

class KDC:
    def __init__(self):
        self.__keys = {}

    def register(self, name):
        key = get_random_bytes(16)
        self.__keys[name] = key
        return key

    def generate_session_key(self, requestor_name, receiver_name):
        if requestor_name in self.__keys and receiver_name in self.__keys:
            # start generate session key
            session_key = get_random_bytes(16)
            print("session key generated as:", session_key)
            # generate IV with 16 byte length
            iv = get_random_bytes(16)
            # encrypt message with receiver's key
            # include iv as first 16 bytes to receiver to decrypt
            enc_msg_receiver = iv + self.encrypt(self.__keys[receiver_name], session_key + requestor_name.encode() ,iv)
            # encrypt previous message with requestor's key
            # encrypt message for requestor will contain 16 bytes IV and input message contain session key for requestor
            enc_msg_requestor = iv + self.encrypt(self.__keys[requestor_name], session_key + enc_msg_receiver ,iv)

            return enc_msg_requestor
        else:
            return None, None

    def encrypt(self, key, msg, iv):
        cipher = AES.new(key, AES.MODE_CBC, iv)
        return cipher.encrypt(pad(msg, AES.block_size))

    def decrypt(self, key, msg, iv):
        cipher = AES.new(key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(msg), AES.block_size)

class KDC_participant:
    name: str
    __kdc: KDC
    __key: bytes
    __session_key: bytes = None

    def __init__(self, name, kdc):
        self.__kdc = kdc
        self.__key = self.__kdc.register(name)
        self.name = name

    def register(self, partner_name):
        # request KDC to generate a session key and msg to participant
        session_key_enc = kdc.generate_session_key(self.name, partner_name)
        iv = session_key_enc[:16]
        raw_data = self.__kdc.decrypt(self.__key, session_key_enc[16:], iv)
        session_key = raw_data[:16]

        tag = raw_data[16:]
        self.__session_key = session_key
        return session_key, tag

    def receive_tag(self, tag):
        iv = tag[:16]
        raw_data = tag[16:]
        raw_data_dec = self.__kdc.decrypt(self.__key, raw_data, iv)
        session_key = raw_data_dec[:16]
        tag = raw_data_dec[16:]
        self.__session_key = session_key
        return raw_data_dec

class Alice(KDC_participant):
    def __init(self, name, kdc):
        super().__init__(name, kdc)

    def request_session_key(self, partner_name):
        return self.register(partner_name)


class Bob(KDC_participant):
    def __init(self, name, kdc):
        super().__init__(name, kdc)

    def receive_tag(self, tag):
        return super().receive_tag(tag)


kdc = KDC()
alice = Alice("Alice", kdc)
bob = Bob("Bob", kdc)

session_key, tag = alice.request_session_key(bob.name)
print("session key:", session_key)
print("tag:", tag)
bob.receive_tag(tag)
print("end!")

register
session key: b'\xbf\xb5\x15\x96\xa3\xba<\nm~/s\xd8\xbd\xa52'
iv: b'\x90\xd3\x8b\xc4Db\xc3\x959Z{~\xe3\x8a"W'
tag: b'\x90\xd3\x8b\xc4Db\xc3\x959Z{~\xe3\x8a"WB\xd4\xea\xc9\x8f\xee\xe1\xea1\xd9\x9e\x83\xee\xd66%\x8cg"\xde^6H\x01=\x98\x18\xa0ci\x13\xca'
iv: b'\x90\xd3\x8b\xc4Db\xc3\x959Z{~\xe3\x8a"W'
session key: b'\xbf\xb5\x15\x96\xa3\xba<\nm~/s\xd8\xbd\xa52'
tag: b'\x90\xd3\x8b\xc4Db\xc3\x959Z{~\xe3\x8a"WB\xd4\xea\xc9\x8f\xee\xe1\xea1\xd9\x9e\x83\xee\xd66%\x8cg"\xde^6H\x01=\x98\x18\xa0ci\x13\xca'
iv: b'\x90\xd3\x8b\xc4Db\xc3\x959Z{~\xe3\x8a"W'
raw data receive:  b'\xbf\xb5\x15\x96\xa3\xba<\nm~/s\xd8\xbd\xa52Alice'
session key b'\xbf\xb5\x15\x96\xa3\xba<\nm~/s\xd8\xbd\xa52'
tag b'Alice'
end!


**Exercise 4**
<br/>
Trusted Third Party (TTP) might be involved in a fair non-repudiation protocol in different extents

- **In-line TTP** acts as an intermediary between the originator and the recipient
  and intervenes directly in a non-repudiation service (e.g. the protocol in Slide
  36).
- **On-line TTP** is actively involved in every instance of a non-repudiation service (e.g. the protocol in Slide 38).
- **Off-line TTP** supports non-repudiation without being involved in each instance of a service

Design (or find in the literature, e.g., in IEEE CSF 1997, “An Efficient Non repudiation Protocol”) a fair non-repudiation protocol using an **off-line TTP** which
does not need to be involved unless the originator or the recipient misbehaves.
